# 고급 튜토리얼: YOLOv8/세분화 + 커스텀 캘리브레이션 로더

# DEEPX DX-COM 튜토리얼 (Ubuntu 24.04 + JupyterLab)
이 노트북은 DEEPX **DX-COM v2.0.0**을 사용하여 ONNX 모델을 **.dxnn**으로 컴파일하는 실습을 단계별로 안내합니다.
**중요:** DX-COM 패키지(tar.gz)를 이미 이 작업 디렉터리에 내려받았다고 가정합니다.
압축을 풀면 `./dx_com/` 폴더가 생기며, 그 안에 `dx_com` 실행파일과 샘플이 포함되어 있습니다.

**학습 목표**
- YOLOv8(탐지/세분화) ONNX 내보내기 및 컴파일
- CSV 기반 **커스텀 로더**로 캘리브레이션 수행
- 파이프라인 자동화 아이디어


## 1) 의존 패키지 설치 (Ultralytics, pandas 등)

In [ ]:
!sudo apt-get update -y && sudo apt-get install -y --no-install-recommends libgl1-mesa-glx libglib2.0-0
!pip install --quiet torch torchvision ultralytics pandas Pillow onnx

# DX-COM 패키지 해제 (이미 해제했다면 생략)
!ls -1 *.tar.gz || true
!tar -xzf dx_com_M1_v2.0.0.tar.gz || echo '이미 해제했거나 파일명을 맞춰주세요.'

## 2) YOLOv8 세분화 모델 ONNX로 내보내기

In [ ]:
from ultralytics import YOLO

# Nano 세분화 모델 (용량/속도 유리)
model = YOLO("yolov8n-seg.pt")  # 없으면 자동 다운로드
onnx_out = model.export(format="onnx", opset=13)
print("✅ YOLOv8-seg ONNX:", onnx_out)

# 생성 파일 확인
!ls -lh | grep yolov8n-seg || true

## 3) YOLOv8 JSON 구성 (640 입력, 간단 전처리)

In [ ]:
import json, os

y8_cfg = {
  "inputs": {"images": [1,3,640,640]},
  "calibration_num": 5,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./yolov5/data/images",   # 예시 이미지 재활용
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 640, "height": 640, "interpolation": "LINEAR"}},
      {"div": {"x": 255.0}}
    ]
  }
}
with open("yolov8_seg.json","w") as f:
    json.dump(y8_cfg,f,indent=2)
print("✅ JSON 저장: yolov8_seg.json")
os.system("sed -n '1,120p' yolov8_seg.json")

## 4) DX-COM 컴파일 (YOLOv8 세분화)

In [ ]:
# 출력 폴더
!mkdir -p dx_com/output/yolov8n_seg

# onnx 파일 위치 자동 탐색 (현재 폴더 기준)
import os, glob
cands = glob.glob("*.onnx")
onnx_file = None
for c in cands:
    if "yolov8n-seg" in c:
        onnx_file = c
        break
print("찾은 ONNX:", onnx_file)

if onnx_file:
    cmd = f"./dx_com/dx_com -m {onnx_file} -c yolov8_seg.json -o dx_com/output/yolov8n_seg"
    print("실행:", cmd)
    exit_code = os.system(cmd)
    print("DX-COM 종료코드:", exit_code)
else:
    print("yolov8n-seg.onnx 파일을 찾지 못했습니다. 상단 export 단계 확인 필요.")

!ls -lh dx_com/output/yolov8n_seg || true

---
## 5) 커스텀 캘리브레이션 데이터 로더 (CSV → 텐서)
여기서는 **MNIST** 테스트셋 100개를 CSV로 저장하고, 이를 읽는 `CustomDataset`을 작성해 DX-COM에 연결합니다.
- CSV 포맷: `label,p0,p1,...,p783` (28x28=784 픽셀)
- Dataset은 (1,28,28) 텐서를 반환


In [ ]:
# MNIST 100샘플을 CSV로 저장
from torchvision import datasets
import numpy as np, os

mnist = datasets.MNIST(root=".", train=False, download=True)
imgs = mnist.data[:100].numpy()
labels = mnist.targets[:100].numpy()

os.makedirs("custom_loader_example/data", exist_ok=True)
csv_path = "custom_loader_example/data/mnist_in_csv.csv"
with open(csv_path,"w") as f:
    for i in range(len(imgs)):
        row = [int(labels[i])] + imgs[i].reshape(-1).tolist()
        f.write(",".join(map(str,row)) + "\n")
print("✅ CSV 저장:", csv_path)

In [ ]:
# CustomDataset 구현 파일 저장 (dx_com/dataset_module.py)
%%bash
cat > dx_com/dataset_module.py << 'PYCODE'
import pandas as pd
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, csv_path="./custom_loader_example/data/mnist_in_csv.csv", height=28, width=28):
        self.data = pd.read_csv(csv_path, header=None)
        self.height = height
        self.width = width
        self.transform = transforms.ToTensor()  # 0~1 float 텐서

    def __len__(self):
        return len(self.data.index)

    def __getitem__(self, index):
        pix = np.array(self.data.iloc[index][1:], dtype=np.uint8).reshape(self.height, self.width)
        img = Image.fromarray(pix, mode='L')
        return self.transform(img)  # (1,28,28)
PYCODE

# 확인
!sed -n '1,120p' dx_com/dataset_module.py

## 6) 간단한 MNIST ONNX 모델 생성 및 JSON(custom_loader) 작성

In [ ]:
import torch, torch.nn as nn, json

class SimpleMNIST(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):          # x: (B,1,28,28)
        x = x.view(x.size(0), -1)  # (B,784)
        x = self.relu(self.fc1(x))
        return self.fc2(x)

m = SimpleMNIST().eval()
dummy = torch.randn(1,1,28,28)
onnx_m = "mnist_model.onnx"
torch.onnx.export(m, dummy, onnx_m, input_names=["image"], output_names=["logits"], opset_version=13)
print("✅ ONNX 저장:", onnx_m)

mnist_json = {
  "inputs": {"image": [1,1,28,28]},
  "calibration_num": 100,
  "calibration_method": "ema",
  "custom_loader": {"package": "dataset_module.CustomDataset"}
}
with open("mnist_model.json","w") as f:
    json.dump(mnist_json,f,indent=2)
print("✅ JSON 저장: mnist_model.json")
!sed -n '1,120p' mnist_model.json

## 7) DX-COM 컴파일 (custom_loader 사용)

In [ ]:
# 출력 폴더
!mkdir -p dx_com/output/mnist_custom

# 컴파일 실행
!./dx_com/dx_com -m mnist_model.onnx -c mnist_model.json -o dx_com/output/mnist_custom || echo '실패 시 dataset_module 위치/경로 확인'

# 결과 확인
!ls -lh dx_com/output/mnist_custom

---
## 8) 파이프라인 자동화 아이디어
- 함수로 묶어: (PyTorch→ONNX)→(JSON작성)→(DX-COM 실행)→(산출물 검증) 일괄 수행
- 모델별 폴더 구조 표준화, 로그/산출물 버전 태깅
- 캘리브레이션 세트 다양화 및 수량 자동 샘플링
